# SAM 2 + NDWI Hybrid Lake Detection

This notebook combines NDWI water detection with SAM 2 boundary refinement for precise glacial lake mapping.

**Approach:**
1. **NDWI pre-filtering**: Identify water candidates using spectral analysis
2. **SAM 2 refinement**: Use SAM 2 to get precise lake boundaries
3. **Validation**: Compare with NDWI-only approach

In [ ]:
# Add SAM 2 to path for direct import (no installation needed)
import sys
sys.path.insert(0, 'sam2_test/segment-anything-2')

import numpy as np
import matplotlib.pyplot as plt
import rasterio
from scipy import ndimage
from skimage import measure
import geopandas as gpd
from PIL import Image
import torch

# SAM 2 imports
import sam2
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

In [ ]:
# Configuration
test_image = "Images_mosaics/langtang/langtang2025_harmonized_mosaics/2025-01-30_composite_mosaic.tif"
slope_mask_shp = "other_data/vect/langtang_slope_lte10_mask_utm_vect_fix.shp"
ndwi_threshold = 0.0
min_candidate_size = 50  # minimum pixels for water candidate

print(f"Using test image: {test_image}")
print(f"NDWI threshold: {ndwi_threshold}")
print(f"Minimum candidate size: {min_candidate_size} pixels")

In [ ]:
# Load and prepare data
print("Loading satellite imagery...")
with rasterio.open(test_image) as src:
    # Read 4-band Planet imagery (RGB + NIR)
    image_data = src.read()  # Shape: (4, height, width)
    transform = src.transform
    crs = src.crs
    
# Convert to standard format
image_data = image_data.transpose(1, 2, 0)  # Shape: (height, width, 4)
height, width, bands = image_data.shape

print(f"Image shape: {image_data.shape}")
print(f"Bands: {bands} (expecting 4: RGB + NIR)")

# Extract individual bands
red = image_data[:, :, 0].astype(np.float32)
green = image_data[:, :, 1].astype(np.float32) 
blue = image_data[:, :, 2].astype(np.float32)
nir = image_data[:, :, 3].astype(np.float32)

# Create RGB image for SAM 2 (expects 3 channels)
rgb_image = image_data[:, :, :3].astype(np.uint8)

print(f"RGB image shape for SAM 2: {rgb_image.shape}")
print(f"RGB value range: {rgb_image.min()} - {rgb_image.max()}")

In [ ]:
# Calculate NDWI
print("Calculating NDWI...")
# NDWI = (Green - NIR) / (Green + NIR)
ndwi = (green - nir) / (green + nir + 1e-8)  # Add small epsilon to avoid division by zero

print(f"NDWI range: {ndwi.min():.3f} to {ndwi.max():.3f}")
print(f"NDWI shape: {ndwi.shape}")

In [ ]:
# Load slope-based analysis mask
print("Loading slope mask...")
slope_gdf = gpd.read_file(slope_mask_shp)

# Create raster mask from shapefile
from rasterio.features import rasterize
from rasterio.transform import from_bounds

# Create mask using the same transform as the image
mask_shapes = [(geom, 1) for geom in slope_gdf.geometry]
analysis_mask = rasterize(
    mask_shapes,
    out_shape=(height, width),
    transform=transform,
    fill=0,
    dtype=np.uint8
)

analysis_mask = analysis_mask.astype(bool)
print(f"Analysis mask shape: {analysis_mask.shape}")
print(f"Analysis area: {analysis_mask.sum()} pixels ({analysis_mask.mean()*100:.1f}% of image)")

In [ ]:
# Step 1: NDWI-based water candidates
print(f"\nStep 1: Finding NDWI water candidates (threshold > {ndwi_threshold})...")

# Apply NDWI threshold within analysis mask
ndwi_candidates = (ndwi > ndwi_threshold) & analysis_mask

print(f"Initial NDWI candidates: {ndwi_candidates.sum()} pixels")

# Clean up candidates with morphological operations
# Opening: remove noise
cleaned_candidates = ndimage.binary_opening(ndwi_candidates, structure=np.ones((3,3)))
# Closing: fill small gaps
cleaned_candidates = ndimage.binary_closing(cleaned_candidates, structure=np.ones((3,3)))

print(f"After morphological cleaning: {cleaned_candidates.sum()} pixels")

# Label connected components
labeled_candidates = measure.label(cleaned_candidates)
regions = measure.regionprops(labeled_candidates)

print(f"Found {len(regions)} candidate regions")

# Filter by size
large_candidates = np.zeros_like(labeled_candidates)
valid_regions = []

for region in regions:
    if region.area >= min_candidate_size:
        large_candidates[labeled_candidates == region.label] = region.label
        valid_regions.append(region)

print(f"Large candidates (>={min_candidate_size} pixels): {len(valid_regions)} regions")
print(f"Total candidate pixels: {(large_candidates > 0).sum()}")

In [ ]:
# Step 2: Initialize SAM 2
print("\nStep 2: Initializing SAM 2...")

# Try different SAM 2 model configurations
sam2_configs = [
    {"config": "sam2_hiera_s.yaml", "checkpoint": "sam2_hiera_small.pt", "name": "small"},
    {"config": "sam2_hiera_b+.yaml", "checkpoint": "sam2_hiera_base_plus.pt", "name": "base_plus"},
    {"config": "sam2_hiera_l.yaml", "checkpoint": "sam2_hiera_large.pt", "name": "large"}
]

sam2_predictor = None

for config in sam2_configs:
    try:
        print(f"Trying SAM 2 {config['name']} model...")
        checkpoint_path = f"sam2_test/segment-anything-2/checkpoints/{config['checkpoint']}"
        model_cfg = config['config']
        
        # Build SAM 2 model
        sam2_model = build_sam2(model_cfg, checkpoint_path, device='cpu')
        sam2_predictor = SAM2ImagePredictor(sam2_model)
        
        print(f"✅ SAM 2 {config['name']} loaded successfully!")
        break
        
    except Exception as e:
        print(f"❌ Failed to load SAM 2 {config['name']}: {e}")
        continue

if sam2_predictor is None:
    print("\n⚠️ SAM 2 not available. Falling back to NDWI-only detection.")
    final_mask = large_candidates > 0
else:
    print("\n🎯 SAM 2 ready for boundary refinement!")

In [ ]:
# Step 3: SAM 2 refinement (if available)
if sam2_predictor is not None:
    print("\nStep 3: SAM 2 boundary refinement...")
    
    # Set the image for SAM 2
    sam2_predictor.set_image(rgb_image)
    
    final_mask = np.zeros_like(ndwi, dtype=bool)
    sam_results = []
    
    print(f"Processing {len(valid_regions)} candidate regions...")
    
    for i, region in enumerate(valid_regions):
        if i >= 10:  # Limit to first 10 regions for demo
            print(f"  Limiting to first 10 regions for demo...")
            break
            
        # Get region centroid as prompt point
        centroid_y, centroid_x = region.centroid
        input_point = np.array([[centroid_x, centroid_y]])
        input_label = np.array([1])  # Positive point
        
        try:
            # Run SAM 2 prediction
            masks, scores, logits = sam2_predictor.predict(
                point_coords=input_point,
                point_labels=input_label,
                multimask_output=True
            )
            
            # Select best mask (highest score)
            best_mask_idx = scores.argmax()
            best_mask = masks[best_mask_idx]
            best_score = scores[best_mask_idx]
            
            # Only use mask if it's within analysis area and has good score
            refined_mask = best_mask & analysis_mask
            
            if refined_mask.sum() > min_candidate_size and best_score > 0.5:
                final_mask |= refined_mask
                sam_results.append({
                    'region_id': i,
                    'centroid': (centroid_x, centroid_y),
                    'score': best_score,
                    'original_area': region.area,
                    'refined_area': refined_mask.sum()
                })
                
            if (i + 1) % 5 == 0:
                print(f"  Processed {i + 1}/{min(len(valid_regions), 10)} regions...")
                
        except Exception as e:
            print(f"  ⚠️ SAM 2 failed for region {i}: {e}")
            continue
    
    print(f"\n✅ SAM 2 refinement complete!")
    print(f"Successful refinements: {len(sam_results)}")
    print(f"Final water pixels: {final_mask.sum()}")
    
else:
    print("\nStep 3: Using NDWI-only detection (SAM 2 not available)")
    final_mask = large_candidates > 0
    sam_results = []

In [ ]:
# Step 4: Results comparison and visualization
print("\nStep 4: Analyzing results...")

# Calculate statistics
total_analysis_pixels = analysis_mask.sum()
ndwi_water_pixels = (large_candidates > 0).sum()
final_water_pixels = final_mask.sum()

print(f"\n📊 Results Summary:")
print(f"Analysis area: {total_analysis_pixels:,} pixels")
print(f"NDWI-only water: {ndwi_water_pixels:,} pixels ({ndwi_water_pixels/total_analysis_pixels*100:.2f}%)")
print(f"Hybrid (SAM2+NDWI) water: {final_water_pixels:,} pixels ({final_water_pixels/total_analysis_pixels*100:.2f}%)")

if sam2_predictor is not None:
    difference = final_water_pixels - ndwi_water_pixels
    print(f"Difference: {difference:+,} pixels ({difference/ndwi_water_pixels*100:+.1f}% change)")
    
    if sam_results:
        avg_score = np.mean([r['score'] for r in sam_results])
        print(f"Average SAM 2 confidence: {avg_score:.3f}")

In [ ]:
# Visualization
print("\nCreating visualization...")

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('SAM 2 + NDWI Hybrid Lake Detection Results', fontsize=16)

# 1. RGB image
axes[0,0].imshow(rgb_image)
axes[0,0].set_title('RGB Satellite Image')
axes[0,0].axis('off')

# 2. NDWI
ndwi_plot = axes[0,1].imshow(ndwi, cmap='RdYlBu', vmin=-0.5, vmax=0.5)
axes[0,1].set_title(f'NDWI (threshold = {ndwi_threshold})')
axes[0,1].axis('off')
plt.colorbar(ndwi_plot, ax=axes[0,1], fraction=0.046)

# 3. Analysis mask
axes[0,2].imshow(analysis_mask, cmap='gray')
axes[0,2].set_title('Analysis Mask (Slope ≤10°)')
axes[0,2].axis('off')

# 4. NDWI candidates
axes[1,0].imshow(rgb_image)
axes[1,0].imshow(large_candidates > 0, alpha=0.6, cmap='Blues')
axes[1,0].set_title(f'NDWI Water Candidates\n({ndwi_water_pixels:,} pixels)')
axes[1,0].axis('off')

# 5. Final result
axes[1,1].imshow(rgb_image)
axes[1,1].imshow(final_mask, alpha=0.6, cmap='Blues')
method_name = 'SAM 2 + NDWI Hybrid' if sam2_predictor is not None else 'NDWI Only'
axes[1,1].set_title(f'{method_name} Result\n({final_water_pixels:,} pixels)')
axes[1,1].axis('off')

# Add SAM 2 prompt points if available
if sam_results:
    for result in sam_results:
        x, y = result['centroid']
        axes[1,1].plot(x, y, 'r+', markersize=8, markeredgewidth=2)

# 6. Comparison
if sam2_predictor is not None:
    # Show difference
    comparison = np.zeros_like(final_mask, dtype=int)
    comparison[large_candidates > 0] = 1  # NDWI only (blue)
    comparison[final_mask & ~(large_candidates > 0)] = 2  # SAM 2 added (green)
    comparison[(large_candidates > 0) & ~final_mask] = 3  # SAM 2 removed (red)
    
    from matplotlib.colors import ListedColormap
    colors = ['white', 'blue', 'green', 'red']
    cmap = ListedColormap(colors)
    
    axes[1,2].imshow(rgb_image)
    comp_plot = axes[1,2].imshow(comparison, alpha=0.6, cmap=cmap, vmin=0, vmax=3)
    axes[1,2].set_title('Method Comparison\nBlue: NDWI, Green: +SAM2, Red: -SAM2')
    axes[1,2].axis('off')
else:
    axes[1,2].text(0.5, 0.5, 'SAM 2 not available\nShowing NDWI-only result', 
                   ha='center', va='center', transform=axes[1,2].transAxes, fontsize=12)
    axes[1,2].axis('off')

plt.tight_layout()
plt.show()

print("\n✅ Analysis complete!")

In [ ]:
# Optional: Save results
print("\nSaving results...")

# Save final water mask
output_mask_path = "sam2_ndwi_hybrid_result.tif"
with rasterio.open(
    output_mask_path, 'w',
    driver='GTiff',
    height=height, width=width,
    count=1, dtype=np.uint8,
    crs=crs, transform=transform
) as dst:
    dst.write(final_mask.astype(np.uint8), 1)

print(f"✅ Water mask saved to: {output_mask_path}")

# Save summary statistics
summary = {
    'image': test_image,
    'ndwi_threshold': ndwi_threshold,
    'analysis_pixels': int(total_analysis_pixels),
    'ndwi_water_pixels': int(ndwi_water_pixels),
    'final_water_pixels': int(final_water_pixels),
    'water_percentage': float(final_water_pixels / total_analysis_pixels * 100),
    'sam2_available': sam2_predictor is not None,
    'sam2_refinements': len(sam_results) if sam_results else 0
}

print(f"\n📄 Summary:")
for key, value in summary.items():
    print(f"  {key}: {value}")

print("\n🎉 SAM 2 + NDWI hybrid detection complete!")